加载云端硬盘，将Google云盘的压缩包在项目空间中解压

参考：https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook

In [ ]:
!unzip "/content/drive/MyDrive/Datasets/dataset_day.zip" -d "/content/"

安装ultralytics（YOLOv8）

In [ ]:
#安装（已完成）
!pip install -U ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

加载数据yaml

后续步骤参考：https://github.com/lyhue1991/eat_yolov8/blob/main/02_YOLOv8%E8%AE%AD%E7%BB%83%E8%87%AA%E5%B7%B1%E7%9A%84%E6%95%B0%E6%8D%AE%E9%9B%86.ipynb

In [ ]:
%%writefile car.yaml
# Ultralytics YOLO 🚀, GPL-3.0 license

path: /content/dataset   # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: image/test # test images (optional)

# Classes
names:
  0: car
  1: bus
  2: truck

In [ ]:
import torch
from torch.utils.data import DataLoader
from ultralytics.cfg import get_cfg
from ultralytics.utils import DEFAULT_CFG, yaml_load
from ultralytics.data.utils import check_cls_dataset, check_det_dataset
from ultralytics.data import build_yolo_dataset, build_dataloader

overrides = {'task':'detect',
             'data':'car.yaml',
             'imgsz':(1280,720),
             'workers':4
            }
cfg = get_cfg(cfg = DEFAULT_CFG,overrides=overrides)
data_info = check_det_dataset(cfg.data)

省略中间步骤，进入定义模型阶段

In [6]:
!cp "/content/drive/MyDrive/Colab Notebooks/yolov8n.pt" "/content/yolov8n.pt"

In [ ]:
from ultralytics.nn.tasks import DetectionModel

model = DetectionModel(cfg = 'yolov8n.yaml', ch=3)
# weights = torch.hub.load_state_dict_from_url('https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt')
weights = torch.load('yolov8n.pt')
model.load(weights['model'])


model.args = cfg
model.nc = data_info['nc']  # attach number of classes to model
model.names = data_info['names']

训练模型阶段

In [ ]:
from ultralytics import YOLO
yolo_model = YOLO('yolov8n.pt')
yolo_model.train(data='car.yaml',epochs=10)

从云端批量下载测试的结果

参考：[colab 从云端批量下载文件到本地\_colab 批量下载文件-CSDN博客](https://blog.csdn.net/jizhidexiaoming/article/details/108302794)

In [ ]:
import os, tarfile

import os
from google.colab import files

#逐个添加文件打包，未打包空子目录。可过滤文件。
#如果只打包不压缩，将"w:gz"参数改为"w:"或"w"即可。
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()

  files.download(output_filename)

make_targz_one_by_one('train_day.tar', '/content/runs/detect/train')

评估部分（直接使用yolo默认的接口，不需要了）